In [5]:
import PortfolioEDA
import pandas as pd
from tabulate import tabulate
import warnings

warnings.filterwarnings('ignore')
tabulatePrint = lambda x: print(tabulate(x, headers='keys', tablefmt='psql'))

In [6]:

portfolioEDA_handle = PortfolioEDA.PortfolioEDA(pd.read_csv('../Data/loantape.csv'), 
                    colNames = {"date":'Snapshotdt', 
                                "loanStatus":'LoanStatus2',
                                "eopBal":'UPB'})

#### Transition Matrix

In [7]:
portfolioEDA_handle.portfolioTrend.generateTransitionMatrix()
portfolioEDA_handle.portfolioTrend.getLatestTransitionMatrix()

LoanStatus2,CO,Current,EarlyDQ,LateDQ,Prepaid
LoanStatus2_Lag1,,,,,
CO,1.000000,0.000000,0.000000,0.000000,0.000000
Current,0.000247,0.900683,0.078077,0.001042,0.019950
EarlyDQ,0.000403,0.325609,0.371572,0.291215,0.011201
LateDQ,0.097143,0.116412,0.080974,0.698031,0.007441
Prepaid,NaN,NaN,NaN,NaN,NaN


#### Stratification

In [9]:
portfolioEDA_handle.portfolioStratification.addMeasure(label = "LoanCount", column = "ApplicationID", calcMethod = "Count", calcHelper = None)

portfolioEDA_handle.portfolioStratification.addMeasure(label = "LoanCount", column = "ApplicationID", calcMethod = "Count", calcHelper = None)
portfolioEDA_handle.portfolioStratification.addMeasure("LoanCount %", "ApplicationID", "Count %", None)
portfolioEDA_handle.portfolioStratification.addMeasure("Financed", "OriginalAmtFinanced", "Sum", None)
portfolioEDA_handle.portfolioStratification.addMeasure("Financed %", "OriginalAmtFinanced", "Sum %", None)

portfolioEDA_handle.portfolioStratification.addMeasure("WAVGLTV", "LTVCore", "Wt_Avg", 'UPB')
portfolioEDA_handle.portfolioStratification.addMeasure("WAVGFICO", "HighFico", "Wt_Avg", 'UPB')
portfolioEDA_handle.portfolioStratification.addMeasure("AvgRate", "CurrentRate", "Wt_Avg", 'UPB')

portfolioEDA_handle.portfolioStratification.addMeasure("PTI", "PTICore", "Wt_Avg", 'UPB')
portfolioEDA_handle.portfolioStratification.addMeasure("DTI", "DTICore", "Wt_Avg", 'UPB')

portfolioEDA_handle.portfolioStratification.addMeasure("BookValue", "BookValue", "Wt_Avg", 'UPB')


In [10]:
# add dimensions
portfolioEDA_handle.portfolioStratification.addDimension(label = "Make", column = "Make", 
                             bucketingRule = None, sortBy = "Financed %", sortAscending = False, topN = 10)

portfolioEDA_handle.portfolioStratification.addDimension("BookNewUsed", "BookNewUsed", None, None, False, None)
portfolioEDA_handle.portfolioStratification.addDimension("BookTier", "BookTier", None, "Financed", False, 5)

portfolioEDA_handle.portfolioStratification.addDimension("LoanStatus", "LoanStatus", None, None, None, None)
portfolioEDA_handle.portfolioStratification.addDimension("LoanTerm", "OriginalTerm", None, None, None, None)

portfolioEDA_handle.portfolioStratification.addDimension("IntRate", "OriginalRate", {"binSize": 0.02, "lower":0.1, "upper":0.26}, None, False, None)
portfolioEDA_handle.portfolioStratification.addDimension("LTVCore", "LTVCore", {"binSize": 0.1}, None, False,  None)
portfolioEDA_handle.portfolioStratification.addDimension("FICO", "HighFico", {"binSize": 30, 'lower': 540, "upper": 720}, None, False,  None)


In [11]:
# generate strats
portfolioEDA_handle.portfolioStratification.generateStrat()

In [12]:
tabulatePrint(portfolioEDA_handle.portfolioStratification.stratsTable['FICO'])

+----------------+-------------+-------------+-------------+---------------+------------------+--------------+-----------+------------+-----------+-----------+----------+-------------+
|                |   LoanCount |   LoanCount |   LoanCount |   LoanCount % |         Financed |   Financed % |   WAVGLTV |   WAVGFICO |   AvgRate |       PTI |      DTI |   BookValue |
|----------------+-------------+-------------+-------------+---------------+------------------+--------------+-----------+------------+-----------+-----------+----------+-------------|
| (-inf, 540.0]  |         558 |         558 |         558 |     0.17007   |      1.20437e+07 |    0.176657  |   1.20459 |    524.255 |  0.173401 | 0.10636   | 0.32485  |     20080.8 |
| (540.0, 570.0] |         745 |         745 |         745 |     0.227065  |      1.57767e+07 |    0.231413  |   1.2039  |    556.471 |  0.170082 | 0.102985  | 0.332297 |     20444.2 |
| (570.0, 600.0] |         756 |         756 |         756 |     0.230418  

In [13]:
tabulatePrint(portfolioEDA_handle.portfolioStratification.stratsTable['Make'])

+----------+-------------+-------------+-------------+---------------+-------------+--------------+-----------+------------+-----------+-----------+----------+-------------+
|          |   LoanCount |   LoanCount |   LoanCount |   LoanCount % |    Financed |   Financed % |   WAVGLTV |   WAVGFICO |   AvgRate |       PTI |      DTI |   BookValue |
|----------+-------------+-------------+-------------+---------------+-------------+--------------+-----------+------------+-----------+-----------+----------+-------------|
| CHEVROLE |         432 |         432 |         432 |     0.131667  | 9.25489e+06 |    0.135751  |   1.19723 |    589.418 |  0.163565 | 0.104641  | 0.32948  |     21550.1 |
| NISSAN   |         419 |         419 |         419 |     0.127705  | 8.54283e+06 |    0.125307  |   1.24569 |    581.17  |  0.166552 | 0.108424  | 0.341989 |     18434.9 |
| FORD     |         257 |         257 |         257 |     0.0783298 | 4.86392e+06 |    0.0713441 |   1.25271 |    588.75  |  0.15

#### Portfolio Trend

In [14]:
# print(portfolioEDA_handle.portfolioTrend.getMeasureTrend('Snapshotdt', 'UPB', 'sum', None, 'eopBal'))
# print(portfolioEDA_handle.portfolioTrend.getMeasureTrend('Snapshotdt', 'HighFico', 'wt_avg', 'UPB', 'FICO'))
# print(portfolioEDA_handle.portfolioTrend.getMeasureTrend('Snapshotdt', 'DQBal', 'sum', None, 'DQBal'))
# print(portfolioEDA_handle.portfolioTrend.getMeasureTrend('Snapshotdt', 'OriginalTerm', 'wt_avg', 'UPB', 'OrigTerm'))
# print(portfolioEDA_handle.portfolioTrend.getMeasureTrend('Snapshotdt', 'RemainingTerm', 'wt_avg', 'UPB', 'RemTerm'))
# print(portfolioEDA_handle.portfolioTrend.getMeasureTrend('Snapshotdt', 'CurrentRate', 'wt_avg', 'UPB', 'IntRate'))

# print(portfolioEDA_handle.portfolioTrend.portfolioCreditStats('cdr', {'defaultBal':'DefaultBal'}))
# print(portfolioEDA_handle.portfolioTrend.portfolioCreditStats('cpr', {'prepayBal':'PrepayBal'}))
# print(portfolioEDA_handle.portfolioTrend.portfolioCreditStats('dq', {'dqBal':'DQBal'}))

                  eopBal
Snapshotdt              
2020-01-31  1.627753e+08
2020-02-29  1.596754e+08
2020-03-31  1.562700e+08
2020-04-30  1.533145e+08
2020-05-31  1.497049e+08
2020-06-30  1.468832e+08
2020-07-31  1.430261e+08
2020-08-31  1.385436e+08
2020-09-30  1.339846e+08
2020-10-31  1.294785e+08
2020-11-30  1.256998e+08
2020-12-31  1.213350e+08
2021-01-31  1.174719e+08
2021-02-28  1.137081e+08
2021-03-31  1.089022e+08
2021-04-30  1.040577e+08
2021-05-31  9.952599e+07
2021-06-30  9.488140e+07
2021-07-31  9.054105e+07
2021-08-31  8.647422e+07
2021-09-30  8.235724e+07
2021-10-31  7.854644e+07
2021-11-30  7.551244e+07
2021-12-31  7.204270e+07
2022-01-31  6.888128e+07
2022-02-28  6.600471e+07
2022-03-31  6.252781e+07
2022-04-30  5.961922e+07
2022-05-31  5.681484e+07
2022-06-30  5.434369e+07
2022-07-31  5.153813e+07
2022-08-31  4.911706e+07
2022-09-30  4.693142e+07
2022-10-31  4.445605e+07
2022-11-30  4.271654e+07
2022-12-31  4.093430e+07
